# Lista 8 z WAS: Testowanie hipotez 

In [2]:
import numpy as np
import pandas as pd
from scipy import stats

## Zadanie 1

In [2]:
n = 47
corr_full = pd.DataFrame(np.array([
    [1, 0.6, -0.36, -0.53],
    [0.6 ,1, -0.46, -0.37],
    [-0.36, -0.46, 1, 0.37],
    [-0.53, -0.37, 0.37, 1]
]))

corr_full.columns = list(range(1,5))
corr_full.index= list(range(1,5))
corr_full

,1,2,3,4
1,1.00,0.60,-0.36,-0.53
2,0.60,1.00,-0.46,-0.37
3,-0.36,-0.46,1.00,0.37
4,-0.53,-0.37,0.37,1.00


### Zadanie 1a: testowanie korelacji cząstkowej 

W podpunkcie a. najpierw wyznaczamy współczynnik $r := \rho_{12|3}$ korzystając tym razem ze wzoru  z zadania 1 z listy 6:
$$\rho_{ij|k} = \frac{\rho_{ij} - \rho_{ik}\rho_{jk}}{\sqrt{(1 - \rho^2_{ik})(1 - \rho_{jk}^2)}}.$$

In [3]:
def get_partial_ij_k(corr: pd.DataFrame, i:int, j:int, k:int) -> float:
    ij = corr.loc[i,j]
    ik = corr.loc[i,k]
    jk = corr.loc[j,k]
    return (ij - ik*jk)/np.sqrt((1 - ik**2)*(1 - jk**2))

r = get_partial_ij_k(corr_full, 1, 2, 3)
print(f"Wspolczynnik korelacji 12|3 wyniosl: {r:.3f}")

Wspolczynnik korelacji 12|3 wyniosl: 0.524


Dla porównania niżej wyliczamy ten sam współczynnik z użyciem funkcji z listy 7 (sposób macierzowy):

In [4]:
import utils_7 as L7
L7.get_partial_ij_ks(corr_full,1,2,[3]).round(3)

0.524

W dalszej częsci podpunktu a. testujemy hipotezę $H_0: r = 0$ przeciwko $H_1: r \neq 0$ z użyciem testu dwustronnego ze statystyką testową
$$T = \hat r \sqrt{\frac{N - q - 2}{1 - r^2}},$$
gdzie $N$ jest liczebnością próby, a $q$ liczbą zmiennych warunkujących. Statystyka ta ma rozkład t-Studenta z $N-2-q$ stopniami swobody (patrz str. 7 wykładu 7). Odrzucimy hipotezę zerową, jeśli wartość bezwzględna T będzie większa niż kwantyl $1 - \frac{\alpha}{2}.$ 

In [5]:
from scipy import stats
N = 47
q = 1
alpha = 0.05

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.t.html
qt = stats.t.ppf(q = 1 - alpha/2, df = N-q-2)
t_stat = r*np.sqrt((N-q-2)/1-r**2)
print(f"Kwantyl: {qt:.3f} | Statystyka testowa: {t_stat:.3f}")

Kwantyl: 2.015 | Statystyka testowa: 3.468


Ponieważ $T > q_t$, odrzucamy hipotezę zerową na zadanym poziomie istotności, tzn. mówimy że istnieje niezerowa korelacja cząstkowa $\rho_{12|3}$.

### Zadanie 1b: testowanie korelacji wielokrotnej

Współczynnik korelacji wielokrotnej wyznaczamy ze wzoru $$R_{1|234} = \sqrt{1 - \frac{det C}{C_{11}}},$$
gdzie $C$ to pełna macierz korelacji a $C_{11}$ dopełnienie algebraiczne elementu $1,1$.

In [6]:
det_C = np.linalg.det(corr_full.to_numpy())
C_11 = (-1)**(1+1) * np.linalg.det(corr_full.to_numpy()[1:,1:]) # bez pierwszego wiersza i kolumny
R = np.sqrt(1 - det_C/C_11)
print(f"Wspolczynnik korelacji R_1|234 wyniosl: {R:.3f}")

Wspolczynnik korelacji R_1|234 wyniosl: 0.686


**UWAGA: R nie przyjmuje wartości ujemnych, więc hipoteza jest jednostronna**.
W dalszej części testujemy hipotezę $H_0: R = 0$ przeciwko $H_1: R> 0$ z użyciem statystyki testowej 
$$F = \frac{N-n}{n-1}\frac{\hat R^2}{1 - \hat R^2},$$
gdzie $N$ to liczebność próby, $n$ - wymiarowość rozkładu, a $\hat R$ estymator próbkowy. Statystyka ta ma rozkład F-Snedecora $F(n-1,N-n)$ (patrz wykład 8).

**Uwaga: Hipoteza jest jednostronny. Dlatego nie dzielimy $\alpha$ przez 2. Ale nie ma to nic wspólnego z tym że rozkład jest niesymetryczny!**

In [7]:
n = 4
F_stat = ((N-n)*R**2) / ((n-1)*(1-R**2))
qf = stats.f.ppf(1 - alpha, n-1, N-n)
print(f"Kwantyl: {qf:.3f} | Statystyka testowa: {F_stat:.3f}")

Kwantyl: 2.822 | Statystyka testowa: 12.717


Statystyka testowa leży w obszarze krytycznym (i to bardzo), więc odrzucamy hipotezę zerową i mówimy że istnieje zależność (liniowa?) pierwszej zmiennej od pozostałych.

## Zadanie 2: testowanie hipotez o średniej rozkładu wielowymiarowego
Testujemy hipotezę $\mu = (0,0)$ przeciwko $\mu \neq (0,0)$ z użyciem statystyki testowej
$$\chi^2 = N D^T \Sigma^{-1} D,$$
gdzie $D = \hat X - \mu_0$ jest różnicą między wektorem średnich próbkowych a średnią w hipotezie zerowej. Statystyka ma rozkład z $n$ stopniami swobody (patrz str. 4 wykładu 7).

W tym zadaniu chcemy ustalić jakie wartości powinny znaleźć się na przeciwprzekątnej macierzy $\Sigma$, żeby odrzucić $H_0$ dla $\alpha = 0.05$. Rozkład $\chi^2$ jest jednostronny, więc nie dzielimy wartości przez dwa. Odrzucimy hipotezę dla wartości statystyki testowej większych od kwantyla $q$ na poziomie $1 - \alpha$ rozkładu $\chi^2_n$, który w naszym wypadku wyniósł:

In [8]:
qchi2 = stats.chi2.ppf(1-0.05, 2) # UWAGA! Statystyka chi2 a nie chi!!
qchi2.round(2)

5.99

Wektor $D = (1,0)^T - (0,0)^T = (1,0)^T$, wtedy statystyka testowa wynosi $$N D^T \Sigma^{-1} D = \frac{N}{3 - \tau^2} = \frac{5}{3 - \tau^2}.$$

Mamy $\frac{5}{3 - \tau^2} > q \iff ... \iff |\tau| > \sqrt{3 - \frac{5}{q}}.$ Wyliczamy wartość wyrażenia po prawej stronie:

In [9]:
np.sqrt(3 - 5/qchi2)

1.4715568280775515

**Odp:** Hipotezę należy odrzucić dla wartości $|\tau| > 1.47$.

## Zadanie 3

In [9]:
X_ = np.array([1,0,2]).T
S = np.array([
    [3, 2, 1],
    [2, 3, 1],
    [1, 1, 4]
])

N = 10
n = 3
alpha = 0.05

In [5]:
def get_T2(X_, S, N, mu_0):
    return N*(X_ - mu_0).T @ np.linalg.inv(S) @ (X_ - mu_0)

## Zadanie 4: testy ze znanym $\Sigma$
Odpowiedzi:
- a. stat = 4.78, q = 5.99, nie odrzucamy
- b. stat = 12, q = 3.84, odrzucamy
- c. stat = 0, q = 3.84, nie odrzucamy
- d. stat = 3, q = 3.84, nie odrzucamy

In [3]:
N = 6
n = 2
Sigma = np.array([[2,-1],[-1,2]])
X_ = np.array([1,1/2]).T
alpha = 0.05

### Zadanie 4a: $\mu = \mu_0$ vs $\mu \neq \mu_0$ dla $= (2, 2/3)^T$
W tym zadaniu używamy testu $\chi^2$, ponieważ macierz kowariancji jest znana. Zauważmy, że do obliczenia statystyki testowej możemy użyć tej samej funkcji co do statystyki Hotellinga - jako `S` podajemy macierz kowariancji a nie jej estymator.

In [6]:
mu_0 = np.array([2, 2/3]).T
chi2_stat = get_T2(X_, Sigma, N, mu_0)
chi2_kwantyl = stats.chi2.ppf(q = 1-alpha, df = n)
print(f"Wartosc statystyki testowej: {chi2_stat:.2f}, kwantyl: {chi2_kwantyl:.2f}")

Wartosc statystyki testowej: 4.78, kwantyl: 5.99


Nie odrzucamy hipotezy zerowej.

### Zadanie 4b
W zadaniach b-d testujemy hipotezy postaci $H_0: a^T\mu = a^T\mu_0$ vs $H_1: a^T\mu \neq a^T\mu_0$ dla różnych wektorów $a$. W zadaniu b mamy $a = (1, 1)^T$ i hipoteza zerowa jest postaci $H_0: C \mu = \mu_0$ dla $C = a$. Statystyka testowa jest postaci
$$T^2 = N (C \bar X - \mu_0)^T (CS^{-1}C)^T (C \bar X - \mu_0)$$
i ma rozkład chi-kwadrat **z k stopniami swobody, gdzie k to pierwszy wymiar C**, ponieważ macierz kowariancji jest znana.

In [39]:
C = np.array([[1, 1]])
k = C.shape[0]
mu_0 = 7/2
T2_stat = N * (C @ X_ - mu_0).T @ np.linalg.inv(C @ Sigma @ C.T) @ (C @ X_ - mu_0)
chi2_kwantyl = stats.chi2.ppf(q = 1-alpha, df = k)
print(f"statystyka: {T2_stat}, kwantyl: {chi2_kwantyl:.2f}")

statystyka: 12.0, kwantyl: 3.84


### Zadanie 4c

In [40]:
C = np.array([[1, -1]])
k = C.shape[0]
mu_0 = 1/2
T2_stat = N * (C @ X_ - mu_0).T @ np.linalg.inv(C @ Sigma @ C.T) @ (C @ X_ - mu_0)
chi2_kwantyl = stats.chi2.ppf(q = 1-alpha, df = k)
print(f"statystyka: {T2_stat}, kwantyl: {chi2_kwantyl:.2f}")

statystyka: 0.0, kwantyl: 3.84


### Zadanie 4d

In [41]:
C = np.array([[1, 0]])
k = C.shape[0]
mu_0 = 2
T2_stat = N * (C @ X_ - mu_0).T @ np.linalg.inv(C @ Sigma @ C.T) @ (C @ X_ - mu_0)
chi2_kwantyl = stats.chi2.ppf(q = 1-alpha, df = k)
print(f"statystyka: {T2_stat}, kwantyl: {chi2_kwantyl:.2f}")

statystyka: 3.0, kwantyl: 3.84


## Zadanie 5
W tym zadaniu podmieniamy kwantyl chi-kwadrat na kwantyl Hotellinga.
Odp:
- statystyki testowe: takie same jak w 4
- kwantyle: a. 17.36, b. 6.61, c. 6.61 d. 6.61
- decyzje: ...


#### 5a

In [47]:
mu_0 = np.array([2, 2/3]).T
chi2_stat = get_T2(X_, Sigma, N, mu_0)
t2_kwantyl = (N - k)  / (N - 1) * k * stats.f.ppf(1-alpha, k, N-k)
print(f"Wartosc statystyki testowej: {chi2_stat:.2f}, kwantyl: {chi2_kwantyl:.2f}")

Wartosc statystyki testowej: 4.78, kwantyl: 3.84


#### 5b

In [48]:
C = np.array([[1, 1]])
k = C.shape[0]
mu_0 = 7/2
T2_stat = N * (C @ X_ - mu_0).T @ np.linalg.inv(C @ Sigma @ C.T) @ (C @ X_ - mu_0)
t2_kwantyl = (N - k)  / (N - 1) * k * stats.f.ppf(1-alpha, k, N-k)
print(f"statystyka: {T2_stat}, kwantyl: {t2_kwantyl:.2f}")

statystyka: 12.0, kwantyl: 6.61


#### 5c

In [49]:
C = np.array([[1, -1]])
k = C.shape[0]
mu_0 = 1/2
T2_stat = N * (C @ X_ - mu_0).T @ np.linalg.inv(C @ Sigma @ C.T) @ (C @ X_ - mu_0)
t2_kwantyl = (N - k)  / (N - 1) * k * stats.f.ppf(1-alpha, k, N-k)
print(f"statystyka: {T2_stat}, kwantyl: {t2_kwantyl:.2f}")

statystyka: 0.0, kwantyl: 6.61


In [50]:
C = np.array([[1, 0]])
k = C.shape[0]
mu_0 = 2
T2_stat = N * (C @ X_ - mu_0).T @ np.linalg.inv(C @ Sigma @ C.T) @ (C @ X_ - mu_0)
t2_kwantyl = (N - k)  / (N - 1) * k * stats.f.ppf(1-alpha, k, N-k)
print(f"statystyka: {T2_stat}, kwantyl: {t2_kwantyl:.2f}")

statystyka: 3.0, kwantyl: 6.61


## Zadanie 6: równość współrzędnych wektora średniej w jednej populacji
czesc 1. Odp: sta: 20,2 kwantyl: 6.4, odrzucamy H0 i wnosimy ze oceny sa rozne 
czesc 2. jednoczesne przedzialy ufnosci: mia-mib \in (-0.526, 0.006), mia-mic, mib-mic, jesli nie zawiera zera to sa rozne

In [51]:
N = 64
X_ = np.array([3.05, 3.31, 2.92]).T
S = np.array([
    [1.28, 1.05, 0.75],
    [1.05, 1.35, 0.93],
    [0.75, 0.93, 1.12]
])

alpha= 0.05


## Zadanie 7: dwie populacje (rozne N)
W danej grupie wektor $X_i = (A_i, B_i, C_i)^T \sim N_3(\mu_i, \Sigma_i)$ reprezentuje wartości trzech składników biochemicznych mózgu u i-tej myszy w tej grupie. Na podstawie danych obliczamy wektor $\hat X$ będący estymatorem parametru $\mu$. Testujemy hipotezę
$$H_0: \mu_C = \mu_T \quad \text{vs } H_1: \mu_C \neq \mu_T,$$
gdzie indeksy oznaczając odpowiednio grupę testową i kontrolną.

In [52]:
N = 24
control = pd.DataFrame({
    "A": [1.21, 0.92, 0.8, 0.85, 0.98, 1.15, 1.10, 1.02, 1.18, 1.09],
    "B": [0.61, 0.43, -.35, 0.48, 0.42, 0.52, 0.5, 0.53, 0.45, 0.4],
    "C": [0.7, 0.71, 0.71, 0.68, 0.71, 0.72, 0.75, 0.7, 0.7, 0.69]
}
)

test = pd.DataFrame({
    "A": [1.4, 1.17, 1.23, 1.19, 1.38, 1.17, 1.31, 1.3, 1.22, 1, 1.12, 1.09],
    "B": [0.5, 0.39, 0.44, 0.37, 0.42, 0.45, 0.41, 0.47, 0.29, 0.3, 0.27, 0.35],
    "C": [0.71, 0.69, 0.7, 0.72, 0.71, 0.7, 0.7, 0.67, 0.68, 0.7, 0.72, 0.73]
}
)

print(control.shape)
print(test.shape)

(10, 3)
(12, 3)


In [53]:
N1 = control.shape[0]
N2 = test.shape[0]
alpha = 0.05
n = 3